`https://tutorials.pytorch.kr/beginner/dcgan_faces_tutorial.html`
# 적대적 생성 신경망(Generative Adversarial Networks)
## GAN??
1. 학습 데이터들의 분포를 학습해, 같은 분포에서 새로운 데이터를 생성할 수 있도록 DL 모델을 학습시키는 프레임워크.
2. **생성자**, **구분자**로 구별되는 두가지 모델을 가지고 잇는 것이 특징.
    > **생성자**의 역할은 실제 이미지로 착각되도록 정교한 이미지를 만드는 것

    > **구분자**의 역할은 이미지를 보고 생성자에 의해 만들어진 이미지인지 실제 이미지인지 알아내는 것
    
    모델을 학습하는 동안
    
    -> **생성자**는 더 진짜 같은 가짜 이미지를 만들어내며 구분자를 속이려함. -> **구분자**는 더 정확히 가짜/진짜 이미지를 구별할 수 있도록 노력함.

    => **생성자**가 학습 데이터들에서 직접 가져온 이미지처럼 보이게 완벽한 이미지를 만들어내고, **구분자**가 생성자에서 나온 이미지를 50%의 확률로 가짜 혹은 진짜로 판별할 때 균형상태에 도달하게 됨.

### 구분자 정의
**$x$: 이미지로 표현되는 데이터**

**$D(x)$: 구분자 신경망, 실제 학습 데이터에서 가져온 $x$를 통과시켜 상수(scalar) 확률값을 결과로 출력함.**

- 이미지 데이터를 다루기에 $D(x)$는 $x$가 학습데이터에서 가져온 것일 때 출력이 크고, $x$일 때 작을 것임.
- $D(x)$는 전통적인 이진 분류기로도 생각될 수 있음.

### 생성자 표기
**$z$: 정규분포에서 뽑은 잠재공간 벡터(laten space vector)** (번역 주. laten space vector는 쉽게 생각해 정규분포를 따르는 n개의 원소를 가진 vector라 볼 수 있음. 다르게 얘기하면 정규분포에서 n개의 원소를 추출한 것과 같음.)

**$G(z)$: $z$벡터를 원하는 데이터 차원으로 대응시키는 신경망으로 둘 수 있음.**
- $G(z)$의 목적은 $p_{data}$에서 얻을 수 있는 학습 데이터들의 분포를 추정하여, 모사한 $p_g$의 분포를 이용해 가짜 데이터들을 만드는 것임.

**$D(G(z))$는 $G$가 출력한 결과물이 실제 이미지일 0~1 사이의 상수의 확률값임.** 
- $D$가 이미지의 참/거짓을 정확히 판별할 확률인 $logD(x)$에서 생성한 이미지를 $D$가
- 가짜로 판별한 확률인 $(log(1-D(G(z))))$를 최소화 시키려는 점에서, $D$와 $G$는 최대최소(minmax) 게임을 하는 것과 같음.

**GAN의 손실함수**


In [ ]:
# GAN 손실 함수
from IPython.display import Image
Image("image/GAN_loss_function.png")

이론적으로는 이 최대최소 게임은 $P_g = P_data$ 이고, 구분자에 입력된 데이터가 1/2의 무작위 확률로 참/거짓이 판별될 때 해답에 이른다.

하지만 GAN의 수렴이론은 연구 중, 현실에서는 최적 상태 도달하지 않는 경우 많음.

## DCGAN??
1. GAN에서 직접적으로 파생된 모델로, 생성자와 구분자에서 합성곱 신경망(convolutoin)과 전치 합성곱 신경망(convolution-transpose)을 사용한 것이 차이점.

### 이전 GAN과 차이점
1. 구분자: convolution, batch norm 계층, LeakyReLU 활성화 함수 사용
- 구분자 입력데이터는 GAN과 같이 3x64x64의 이미지
- 출력값은 입력데이터가 실제 데이터일 0~1 사이의 확률값
2. 생성자: convolutional-transpose 계층, 배치 정규화(batch norm) 계층, ReLU 활성화 함수 사용
- 입력값은 정규분포에서 추출한 잠재공간 벡터 $z$
- 출력값은 3x64x64 RGB 이미지
> 이때, 전치 합성곱 신경망은 잠재공간 벡터로 하여금 이미지와 같은 차원을 같도록 변환시켜주는 역할을 함. (번역 주. 전치 합성곱 신경망은 합성곱 신경망의 반대적인 개념이라 이해하면 쉽다. 입력된 작은 CHW 데이터를 가중치들을 이용해 더 큰 CHW로 업샘플링해주는 계층이다.)


In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 코드 실행결과의 동일성을 위해 무작위 시드를 설정합니다
manualSeed = 999
#manualSeed = random.randint(1, 10000) # 만일 새로운 결과를 원한다면 주석을 없애면 됩니다
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

## 설정값
-  **dataroot** - 데이터셋 폴더의 경로.
-  **workers** - DataLoader에서 데이터를 불러올 때 사용할 쓰레드의 개수.
-  **batch_size** - 학습에 사용할 배치 크기. DCGAN에서는 128을 사용.
-  **image_size** - 학습에 사용되는 이미지의 크기.
   본 문서에서는 64x64의 크기를 기본으로 하나, 만일 다른 크기의 이미지를 사용한다면
   D와 G의 구조 역시 변경되어야... 더 자세한 정보를 위해선
   `이곳 <https://github.com/pytorch/examples/issues/70>`__ 을 확인.
-  **nc** - 입력 이미지의 색 채널 개수. RGB 이미지이기 때문에 3으로 설정.
-  **nz** - 잠재공간 벡터의 원소들 개수.
-  **ngf** - 생성자를 통과할때 만들어질 특징 데이터의 채널 개수.
-  **ndf** - 구분자를 통과할때 만들어질 특징 데이터의 채널 개수.
-  **num_epochs** - 학습시킬 에폭 수. 오래 학습시키는 것이 대부분 좋은 결과를 보이지만, 당연히도 시간이 오래걸리는 것이 단점.
-  **lr** - 모델의 학습률입니다. DCGAN에서 사용된대로 0.0002로 설정.
-  **beta1** - Adam 옵티마이저에서 사용할 beta1 하이퍼파라미터 값. 역시나 논문에서 사용한대로 0.5로 설정.
-  **ngpu** - 사용가능한 GPU의 번호. 0으로 두면 CPU에서 학습하고, 0보다 큰 수로 설정하면 각 숫자가 가리키는 GPU로 학습시킴.


In [ ]:
# 데이터셋의 경로
dataroot = "data/celeba"

# dataloader에서 사용할 쓰레드 수
workers = 2

# 배치 크기
batch_size = 128

# 이미지의 크기. 모든 이미지들은 transformer를 이용해 64로 크기가 통일됨.
image_size = 64

# 이미지의 채널 수로, RGB 이미지이기 때문에 3으로 설정.
nc = 3

# 잠재공간 벡터의 크기 (예. 생성자의 입력값 크기)
nz = 100

# 생성자를 통과하는 특징 데이터들의 채널 크기
ngf = 64

# 구분자를 통과하는 특징 데이터들의 채널 크기
ndf = 64

# 학습할 에폭 수
num_epochs = 5

# 옵티마이저의 학습률
lr = 0.0002

# Adam 옵티마이저의 beta1 하이퍼파라미터
beta1 = 0.5

# 사용가능한 gpu 번호. CPU를 사용해야 하는경우 0으로
ngpu = 1

## 데이터
1. 데이터는 `Celeb-A Faces dataset` <http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html>에서 (img_align_celeba.zip)
2. 다운로드가 끝나면 *celeba*라는 새로운 폴더에 해당 zip 파일을 압축해제
3. 위에서 정의한 *dataroot* 변수에 방금 만든 *celeba* 폴더의 경로 삽입

        /path/to/celeba
            -> img_align_celeba
                -> 188242.jpg
                -> 173822.jpg
                -> 284702.jpg
                -> 537394.jpg
                    ...

> *celeba* 폴더 안에 다시 폴더를 두는 이유는 ImageFolder 클래스가 데이터셋의 최상위 폴더에 서브 폴더를 요구하기 때문임.

In [ ]:
# 학습 데이터 시각화

# 이미지 데이터셋 불러오기
# 데이터셋 만들기
dataset = dset.ImageFolder(root=dataroot,
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))

# dataloase를 정의하기
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                            shuffle=True, num_workers=workers)

# GPU 사용 여부 결정
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# 학습 데이터 이미지들 몇 개만 화면에 띄워보기
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

## 구현
모델의 구현

가중치 초기화->생성자->구분자->손실함수->학습방법에 대해 소개

### 가중치 초기화
- DCGAN 논문에서는 평균이 0, 분산이 0.02인 정규 분포를 이용해서 구분자와 생성자 모두 무작위 초기화를 진행하는 것이 좋다고 함.
- `weights_init` 함수는 매개변수로 모델을 입력받아, 모든 합성곱 계층, 전치 합성곱 계층, 배치 정규화 계층을 위에서 말한 조건대로 가중치들을 다시 초기화 시킴.
> 이 함수는 모델이 만들어지면 바로 적용을 시키게 됨.

In [ ]:
# netG와 netD에 적용시킬 커스텀 가중치 초기화 함수.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### 생성자
생성자 $G$는 잠재 공간 벡터 $z$를, 데이터 공간으로 변환시키도록 설계되었음.

데이터는 이미지이기에 $z$를 데이터 공간으로 변환한다는 뜻은 학습 이미지와 같은 사이즈를 가진 RGB 이미지를 생성하는 것과 같음.

`예시로 3x64x64`
- 실제 모델에서는 stride 2를 가진 전치 합성곱 계층들을 이어서 구성하는데, 각 전치 합성곱 계층 하나 당 2차원 배치 정규화 계층과 relu 활성함수를 한 쌍으로 묶어서 사용함.
- 생성자의 마지막 출력 계층에서는 데이터를 tanh 함수에 통과시키는데, 이는 출력 값을 [-1, 1] 사이의 범위로 조정하기 위해서임.
- 이때, 배치 정규화 계층을 주목, DCGAN 논문에 의하면 이 계층이 경사하강법(gradient-descent)의 흐름에 중요한 영향을 미치는 것으로 알려져 있음.

#### DCGAN 논문에서 가져온 생성자 모델 아키텍쳐

In [ ]:
# DCGAN 논문에서 가져온 생성자 모델 아키텍쳐
from IPython.display import Image
Image("image/dcgan_generator.png")

설정값 섹션에서 정의한 값들이 (nz, ngf, nc) 생성자 모델 아키텍쳐에 어떻게 영향을 주는지 주목.
1. nz는 z 입력 벡터의 길이
2. ngf는 생성자를 통과하는 특징 데이터의 크기
3. nc는 출력 이미지의 채널 개수

(RGB 이미지이기 때문에 nc는 3으로 설정)

#### 생성자 코드

In [ ]:
# 생성자 코드
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 입력데이터 Z가 가장 처음 통과하는 전치 합성곱 계층
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 위의 계층을 통과한 데이터의 크기. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # 위의 계층을 통과한 데이터의 크기. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

생성자의 인스턴스를 만들고 `weights_init`함수를 적용시킬 수 있다. 모델의 인스턴스를 출력해서 생성자가 어떻게 구성되어있는지 확인.

In [ ]:
# 생성자를 만듦
netG = Generator(ngpu).to(device)

# 필요한 경우 multi-gpu를 설정
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# 모든 가중치의 평균을 0, 분산을 0.02으로 초기화 하기 위해 weight_init 함수를 적용시킴
netG.apply(weights_init)

# 모델의 구조 출력
print(netG)

### 구분자

구분자 $D$는 3x64x64 이미지를 받아, Conv2d, BatchNorm2d, LeakyReLU 계층을 통과시켜 데이터를 가공시키고, 마지막 출력에서 Sigmoid 함수를 이용하여 0~1 사이의 확률값으로 조정함.

이 아키텍쳐는 필요한 경우 더 다양한 레이어를 쌓을 수 있지만 배치 정규화, LeakyReLU, 특히 보복이 있는 (strided) 합성곱 계층을 사용하는 것에는 이유가 있음.
> DCGAN 논문에서는 보폭이 있는 합성곱 계층을 사용하는 것이 신경망 내에서 스스로의 풀링(Pooling) 함수를 학습하기 때문에, 데이터를 처리하는 과정에서 직접적으로 풀링 계층(MaxPool or AvgPooling)을 사용하는 것보다 더 유리하다고 함.

> 또한 배치 정규화와 leaky relu 함수는 학습과정에서 $G$ 와 $D$ 가 더 효과적인 경사도(gradient)를 얻을 수 있음.

#### 구분자 코드

In [ ]:
# 구분자 코드

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 입력 데이터의 크기는 (nc) x 64 x 64 입니다
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # 위의 계층을 통과한 데이터의 크기. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

생성자에 한 것처럼 구분자의 인스턴스를 만들고, ``weights_init`` 함수를 적용시킨 다음, 모델의 구조를 출력해볼 수 있음.

In [ ]:
# 구분자를 만듦
netD = Discriminator(ngpu).to(device)

# 필요한 경우 multi-gpu를 설정.
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# 모든 가중치의 평균을 0, 분산을 0.02로 초기화 하기 위해
# weight_init 함수를 적용.
netD.apply(weights_init)

# 모델의 구조를 출력
print(netD)

### 손실함수와 옵티마이저
$D$와 $G$의 설정을 끝냈으니, 손실함수와 옵티마이저를 정해서 학습을 구체화.

> 손실함수: Binary Cross Entropy loos(BCELoss)를 사용

\begin{align}\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad l_n = - \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]\end{align}


이때, 위의 함수가 로그함수 요소를 정의한 방식을 주의깊게 보면

(예. $log(D(x))$ 와 $log(1-D(G(z)))$). 우린 $y$ 을 조정을 조정하여, BCE 함수에서 사용할 요소를 고를 수 있음.
이 부분은 이후에 다루지만, 어떻게 $y$ 를 이용하여 원하는 요소들만 골라낼 수 있는지 이해하는 것이 먼저. (예. GT labels).

<br><br>

다음으로, 참 라벨(혹은 정답)은 1로 두고 거짓 라벨(혹은 오답)은 0으로 뒀을 때,

각 라벨의 값을 정한 건 GAN 논문에서 사용된 값들로 GAN을 구성할 때의 관례라 할 수 있음. 방금 정한 라벨 값들은 추후에 손실값을 계산하는 과정에서 사용될 것임.

<br><br>

마지막으로, 서로 구분되는 두 옵티마이저를 구성.

하나는 $D$를 위한 것, 하나는 $G$를 위한 것.

DCGAN에 서술된 대로, 두 옵티마이저는 모두 Adam을 사용하고 학습률은 0.0002, Beta1은 0.5로 설정

추가적으로, 학습이 되는 동안 생성자의 상태를 알아보기 위해 프로그램이 끝날 때까지 고정된 잠재공간 벡터를 생성(ex. fixed_noise)

이 벡터들 역시 가우시안 분포에서 추출함. 학습 과정을 반복하면서 $G$에 주기적으로 같은 잠재 공간 벡터를 입력하면, 그 출력값을 기반으로 생성자의 상태를 확인할 수 있음.

In [ ]:
# BCELoss 함수의 인스턴스를 생성
criterion = nn.BCELoss()

# 생성자의 학습상태를 확인할 잠재 공간 벡터를 생성
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# 학습에 사용되는 참/거짓의 라벨을 정함
real_label = 1.
fake_label = 0.

# G와 D에서 사용할 Adam옵티마이저를 생성
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

### 학습
최종, GAN 프레임워크에 필요한 모든 부분은 설정했으니, 실제 모델을 학습시키는 방법에 대한 부분

※ GAN을 학습시키는 건 관례적인 기술들의 집합이기 때문에 잘못된 하이퍼파라미터의 설정은 모델의 학습을 망가뜨릴 수 있음.
> goodfellow의 논문에서 서술된 Algorithm 1을 기반으로 ganhacks에서 사용된 몇가지 괜찮은 테크닉을 더해서 사용됨!

<br><br>
*의도는 "진짜 혹은 가짜 이미지를 구성" 하고, $logD(G(z))$를 최대화하는 $G$의 목적함수를 최적화 시키는 것.*

학습과정은 크게 두가지로 나뉨
- part1: 구분자 업데이트
- part2: 생성자 업데이트

**part1- 구분자의 학습**
- 구분자의 목적: 주어진 입력값이 진짜인지 가짜인지 판별하는 것.
    - Goodfellow: "변화도(gradient)를 상승(ascending) 시키며 훈련"
    - 실전적으로: $log(D(x)) + log(1-D(G(z)))$ 를 최대화시키는 것
- ganhacks에서 미니 배치를 분리하여 사용한 개념을 가져와서 두가지 스텝으롭 분리해서 계산
    1. 진짜 데이터들로만 이루어진 배치를 만들어 $D$에 통과시키고, 그 출력값으로 $(log(D(x)))$의 손실값을 계산하고, 역전파 과정에서의 변화도들을 계산함
    2. 가짜 데이터들로만 이루어진 배치를 만들어 $D$에 통과시키고, 그 출력값으로 $(log(1-D(G(x))))$의 손실값을 계산하고, 역전파 변화도를 구함
    > 이때, 두가지 스텝에서 나오는 변화도들은 축적(accumulate) 시켜야 함.
    
    변화도를 구했으니, 옵티머이저를 사용(파이토치의 함수를 호출)

**part2- 생성자의 학습**
- 생성자: (original GAN 논문) $log(1-D(G(z)))$을 최소화 시키는 방향으로 학습함.
    - Goodfellow: "위의 방식은 충분한 변화도를 제공하지 못함" (특히 학습 초기에 문제)

- 해결 방법: $log(D(G(z)))$를 최대화 하는 방식으로 바꿔서 학습
    - part1에서 한대로 구분자를 이용해 생성자의 출력값을 판별해주고, 진짜 라벨값을 이용해 $G$의 손실값을 구해줌, 그러면 구해진 손실값으로 변화도를 구하고, 최종적으로 옵티마이저를 이용해 G의 가중치들을 업데이트 시켜주면 됨.
    > 언뜻 보면 생성자가 만들어낸 가짜 이미지에 진짜 라벨을 사용하는 것이 직관적으로 위배가 되겠지만, 라벨을 바꿔줘서 $log(x)$라는 BCELoss의 일부분을 사용할 수 있게 함
    
    (앞서, BCELoss에서 라벨을 이용해 원하는 로그 계산 요소를 고를 수 있음을 알아봄.)

- 마지막으로 $G$의 훈련 상태를 알아보기 위하여, 몇가지 통계적인 수치들과, fixed_noise를 통과시킨 결과를 화면에 출력하는 코드를 추가.

    <통계적인 수치들>

    -  **Loss_D** - 진짜 데이터와 가짜 데이터들 모두에서 구해진 손실값. ($log(D(x)) + log(1 - D(G(z)))$).
    -  **Loss_G** - 생성자의 손실값. $log(D(G(z)))$
    -  **D(x)** - 구분자가 데이터를 판별한 확률값입니다. 처음에는 1에 가까운 값이다가,
    G가 학습할수록 0.5값에 수렴하게 됩니다.
    -  **D(G(z))** - 가짜데이터들에 대한 구분자의 출력값입니다. 처음에는 0에 가까운 값이다가,
    G가 학습할수록 0.5에 수렴하게 됩니다

In [ ]:
# 학습 과정

# 학습상태를 체크하기 위해 손실값들을 저장
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# 에폭(epoch) 반복
for epoch in range(num_epochs):
    # 한 에폭 내에서 배치 반복
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) D 신경망을 업데이트: log(D(x)) + log(1 - D(G(z)))를 최대화
        ###########################
        ## 진짜 데이터들로 학습을
        netD.zero_grad()
        # 배치들의 사이즈나 사용할 디바이스에 맞게 조정
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label,
                           dtype=torch.float, device=device)
        # 진짜 데이터들로 이루어진 배치를 D에 통과
        output = netD(real_cpu).view(-1)
        # 손실값을 구함
        errD_real = criterion(output, label)
        # 역전파의 과정에서 변화도를 계산
        errD_real.backward()
        D_x = output.mean().item()

        ## 가짜 데이터들로 학습
        # 생성자에 사용할 잠재공간 벡터를 생성
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # G를 이용해 가짜 이미지를 생성
        fake = netG(noise)
        label.fill_(fake_label)
        # D를 이용해 데이터의 진위를 판별
        output = netD(fake.detach()).view(-1)
        # D의 손실값을 계산
        errD_fake = criterion(output, label)
        # 역전파를 통해 변화도를 계산. 이때 앞서 구한 변화도에 더함(accumulate)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # 가짜 이미지와 진짜 이미지 모두에서 구한 손실값들을 더함
        # 이때 errD는 역전파에서 사용되지 않고, 이후 학습 상태를 리포팅(reporting)할 때 사용함
        errD = errD_real + errD_fake
        # D를 업데이트
        optimizerD.step()

        ############################
        # (2) G 신경망을 업데이트: log(D(G(z)))를 최대화
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # 생성자의 손실값을 구하기 위해 진짜 라벨을 이용
        # 우리는 방금 D를 업데이트했기 때문에, D에 다시 가짜 데이터를 통과시킴
        # 이때 G는 업데이트되지 않았지만, D가 업데이트 되었기 때문에 앞선 손실값가 다른 값이 나오게 됨
        output = netD(fake).view(-1)
        # G의 손실값을 구함
        errG = criterion(output, label)
        # G의 변화도를 계산함
        errG.backward()
        D_G_z2 = output.mean().item()
        # G를 업데이트
        optimizerG.step()

        # 훈련 상태를 출력
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # 이후 그래프를 그리기 위해 손실값들을 저장
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # fixed_noise를 통과시킨 G의 출력값을 저장
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

### 결과
1. $G$와 $D$의 손실값들이 어떻게 변했는지
2. 매 에폭마다 fixed_noise를 이용해 $G$가 만들어낸 이미지들
3. 학습이 끝난 $G$가 만들어낸 이미지와 진짜 이미지 비교

**학습하는 동안의 손실값들**

아래는 D와 G의 손실값들을 그래프로 그린 모습

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

#### $G$의 학습 과정 시각화
매 에폭마다 fixed_noise를 이용해 생성자가 만들어낸 이미지를 저장했었음.

저장한 이미지들을 애니메이션 형식으로 확인.

play를 누르면 애니메이션 실행됨.

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

#### 진짜 이미지 vs 가짜 이미지

In [ ]:
# dataloader에서 진짜 데이터들을 가져옵니다
real_batch = next(iter(dataloader))

# 진짜 이미지들을 화면에 출력합니다
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# 가짜 이미지들을 화면에 출력합니다
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()